In [ ]:
# 3.0 LLMs and Chat Models
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

llm.predict("How many planets are there?")
chat.predict("How many planets are there?")

In [ ]:
# 3.1 Predict Messages with Chat Model

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

chat = ChatOpenAI(temperature=0.1)

messages = [
    SystemMessage(content="You are Madalena Bay. Reply as if you were her."),
    AIMessage(content="It's me, Imaginal Disk"),
    HumanMessage(content="What songs you are influenced by?")
]

chat.predict_messages(messages)

In [ ]:
# 4.1 PromptTemplate without from_template

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI()

template = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

template.format(country="France")

In [ ]:
# 3.2 Customize prompt with Prompt Template

from langchain.chat_models import  ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What songs did Magdalena Bay listen when making {track}?"
)
prompt = template.format(track="Imaginal Disk")

chat.predict(prompt)

In [ ]:
# 3.2 Customize prompt with Chat Prompt Template

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    (
        "human",
        "What is the distance between {country_a} and {country_b}. Also, what is your name?",
    ),
])
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

In [ ]:
# 3.3 Concept of OutputParser

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("It's ,you, the,  purest,you")

In [ ]:
# 3.3 Apply OutputParser

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system",
    """
     You are a list generating machine. 
     Answer question with these conditions.
     - COMMA SEPERATED LIST
     - List have maximum {max_items} items.
     - All characters in lowercase
    """),
    ("human","{question}")
])
prompt = template.format_messages(
    max_items=5,
    question="What are the pokemons?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

In [ ]:
# 3.3 OutputParser and LCEL

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)
template = ChatPromptTemplate.from_messages([
    (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
    ),
    ("human", "{question}")
])
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question":"What are the pokemons?"})

In [ ]:
# 3.4 Chaining Chains

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # listening to events on LLM
)
original_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a pro baker. You create a easy to follow recipe that anyone can succeed with."),
    ("human", "I want to bake {bread_type}.")
])
original_chain = original_prompt | chat

modify_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a Korean baker. You change the original baking recipe to Koreans love. You add and adjust some ingredients. If you don't know how to convert, just say so."),
    ("human", "{recipe}")
])
modify_chain = modify_prompt | chat

final_chain = {"recipe": original_chain} | modify_chain
# modify_chain.invoke({"recipe": original_chain_result})

final_chain.invoke({"bread_type": "Country Bread"})

In [ ]:
# 4.1 FewShotPromptTemplate: Give examples to the model how to answer.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# -- 1. Get examples from database
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat = ChatOpenAI()

# -- 2. Create prompt to format examples
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# -- 3. Give examples to the FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?", # same format as the question of examples
    input_variables=["country"],
)

chain = prompt | chat
chain.invoke({"country":"Germany"})

In [ ]:
# 4.2 FewShotChatMessagePromptTemplate

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat = ChatOpenAI(temperature=0.1)

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. You give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat
chain.invoke({"country":"Turkey"})

In [ ]:
# 4.3 LengthBasedExampleSelector: 비용이나 모델 제약 때문에 모델에 보낼 examples를 select 해야할 때

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat = ChatOpenAI(temperature=0.1)

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

In [ ]:
# 4.3 Making your own example selector: Random Example Selector
'''
ex)
- if user is logged-in or not
- language of the user 
'''

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples: list[dict[str, str]]):
        self.examples = examples

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
    
    def add_example(self, example): # add_example to already existing examples list
        self.examples.append(example)
    
chat = ChatOpenAI(temperature=0.1)
example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")
example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="What do you know about {country}?",
    input_variables=["country"],
)
prompt.format(country="Brazil")


In [ ]:
# 4.4 Serialization: Load PromptTemplates from disk

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt

chat = ChatOpenAI()
json_prompt = load_prompt("./prompt.json")
yaml_prompt = load_prompt("./prompt.yaml")

json_prompt.format(country="Germany")

'What is the capital of Germany'

In [ ]:
# 4.4 Composition: 여러 prompts 합치기

from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

chain = full_prompt | chat
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

In [ ]:
# 4.5 Caching: Save responses from LM

from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache # memory에 caching하기 때문에 notebook을 재시작하면 refreshed.
from langchain.cache import SQLiteCache # Save in database
from langchain.chat_models import ChatOpenAI

set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
chat = ChatOpenAI(temperature=0.1)

chat.predict("How to make Italian Pasta?")

In [ ]:
# 4.5 SetDebug: 무슨 일을 하고 있는지 로그 기록을 보여준다.

from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache
from langchain.chat_models import ChatOpenAI

set_llm_cache(InMemoryCache())
set_debug=True

chat = ChatOpenAI(temperature=0.1)

chat.predict("How to make Italian Pasta?")

In [ ]:
# 4.6 지출 비용 알리미

from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju?")
    b = chat.predict("What is the recipe for sourdough?")
    print(usage)
    print(usage.total_cost)
    print(usage.total_tokens)
    print(usage.prompt_tokens)

In [ ]:
# 4.6 Serialization: Save models
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    model="gpt-5-nano",
    max_tokens=450,
)

chat.save("model.json")

In [ ]:
# 4.6 Serialization: Load models
from langchain.llms.loading import load_llm

chat = load_llm("model.json")